# Qiskit Coding Tasks: Entanglement & Bell's Theorem

This notebook contains three coding exercises designed to help you experimentally verify the concepts from your notes and worksheet. You will need `qiskit` and `numpy` installed.

`pip install qiskit qiskit-aer numpy`

## Setup: Imports for all tasks

In [2]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.quantum_info import Statevector
from numpy import pi
import random

# Set up the simulator
simulator = AerSimulator()

--- 

## Task 1: Tensor Products & Separability


### 1. Engage (Why?)
You've calculated tensor products by hand. Now, let's see how a quantum computer simulator handles them instantly. We'll verify your hand-written answers from the worksheet.

### 2. Explore (Play)
Let's start with the simplest case. We'll define two separate, simple states: `|0⟩` and `|1⟩` and then combine them.

In [3]:
# Create a statevector for |0>
sv_0 = Statevector([1, 0])

# Create a statevector for |1>
sv_1 = Statevector([0, 1])

# Combine them using the tensor product
sv_01 = sv_0.tensor(sv_1)

# Print the result
print("State |01>:")
print(sv_01)

State |01>:
Statevector([0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j],
            dims=(2, 2))


**Run this code.** What state does this correspond to in Dirac notation? ($|00\rangle, |01\rangle, |10\rangle,$ or $|11\rangle$?)

### 3. Explain (The Concept)
The `Statevector.tensor()` function performs the tensor product. The state `sv_01` is now a 4-dimensional vector representing the combined two-qubit system.
A state is **separable** if it can be created by `A.tensor(B)`.
A state is **entangled** if it *cannot* be.

### 4. Elaborate (Apply)

- Create the statevectors for $|+\rangle$ and $|-\rangle$.
- Calculate their tensor product $|\psi\rangle = |+\rangle \otimes |-\rangle$.
- Print the final statevector. Does it match your manual calculation?

In [ ]:
# --- Your Code Here --- 

# Recall: |+> = 1/sqrt(2) * [1, 1]
# Recall: |-> = 1/sqrt(2) * [1, -1]

# Create sv_plus
sv_plus = Statevector([1/np.sqrt(2), 1/np.sqrt(2)])

# Create sv_minus
sv_minus = Statevector([1/np.sqrt(2), -1/np.sqrt(2)])

# Calculate the tensor product
sv_psi = sv_plus.tensor(sv_minus)

print("\nState |+> tensor |->:")
print(sv_psi)

# --- End Your Code ---

### 5. Evaluate (Challenge)

- Manually create the statevector for $|\Psi\rangle = \frac{1}{\sqrt{2}}|00\rangle + \frac{1}{\sqrt{2}}|10\rangle$.
- In your worksheet, you showed this state is separable by factoring it into $|+\rangle \otimes |0\rangle$.
- In the code cell below, we will *verify* this factorization. We'll create the $|+\rangle|0\rangle$ state and use `Statevector.equiv()` to see if it's identical to $|\Psi\rangle$.

In [6]:
# --- Your Code Here --- 

# |Psi> = [1/sqrt(2), 0, 1/sqrt(2), 0]
sv_psi_2 = Statevector([1/np.sqrt(2), 0, 1/np.sqrt(2), 0])

# 1. Create the presumed factored states
sv_plus = Statevector([1/np.sqrt(2), 1/np.sqrt(2)])
sv_0 = Statevector([1, 0])

# 2. Tensor them together |+0>
sv_factored = sv_plus.tensor(sv_0)

# 3. Check if the statevectors are equivalent.
# We can use the .equiv() method, which checks for equality 
# while ignoring global phase (which is good practice).
is_sep = sv_psi_2.equiv(sv_factored)

print(f"\nOur factored state from coding is: {sv_factored}")
print(f"Is this equivalent to |Psi>? {is_sep}")
print(f"\nTherefore, is the state |Psi> separable? {is_sep}")

# --- End Your Code ---


Our factored state from coding is: Statevector([0.70710678+0.j, 0.        +0.j, 0.70710678+0.j,
             0.        +0.j],
            dims=(2, 2))
Is this equivalent to |Psi>? True

Therefore, is the state |Psi> separable? True


--- 

## Task 2: Formal Matrix Construction of the Bell State


### 1. Engage (Why?)
The $|\Phi^+\rangle$ Bell state is the most famous entangled state. You've seen the math in the notes. Now, you'll build the exact quantum circuit that creates it.

### 2. Explore (Play)
Let's build the circuit step-by-step. First, we'll only apply the Hadamard gate to `q0` (qubit 0). Also, try to calculate by hand side by side with your code

In [ ]:
# Create a 2-qubit circuit
qc = QuantumCircuit(2)

# Apply H-gate to q0
qc.h(0)

# See what the circuit looks like
print("Circuit - Step 1:")
print(qc.draw())

# --- Get the Statevector --- 
# We can use the .save_statevector() instruction to convert the circuit to a statevector
qc.save_statevector()

# Transpile and run the circuit
compiled_circuit = transpile(qc, simulator)
job = simulator.run(compiled_circuit)
result = job.result()

# Get the final statevector
sv_step1 = result.get_statevector()
print(f"\nStatevector after Step 1: {sv_step1}")

Circuit - Step 1:
     ┌───┐
q_0: ┤ H ├
     └───┘
q_1: ─────
          

Statevector after Step 1: Statevector([0.70710678+0.j, 0.70710678+0.j, 0.        +0.j,
             0.        +0.j],
            dims=(2, 2))


**Run this code.** Compare the output `Statevector after Step 1` to your answer calculated by hand. Does it match?

### 3. Explain (The Concept)
You've created the state $|0\rangle|+\rangle$, which is `[0.707, 0.707, 0, 0]`. This is still a separable state. To entangle them, we need the `CNOT` gate, which makes the state of `q1` *dependent* on the state of `q0`.

### 4. Elaborate (Apply)
Now, add the `CNOT` gate to your circuit, with `q0` as the control and `q1` as the target. Then, get the final statevector.

In [10]:
# --- Your Code Here --- 

# Create a new 2-qubit circuit
qc_bell = QuantumCircuit(2)

# Step 1: Apply H-gate to q0
qc_bell.h(0)

# Step 2: Apply CNOT-gate (q0=control, q1=target)
qc_bell.cx(0, 1)

# Draw the final circuit
print("Circuit - Final Bell State:")
print(qc_bell.draw())

# Get the statevector
qc_bell.save_statevector()
compiled_bell_circuit = transpile(qc_bell, simulator)
job_bell = simulator.run(compiled_bell_circuit)
result_bell = job_bell.result()

# Get the final statevector
sv_final = result_bell.get_statevector()
print(f"\nFinal Bell Statevector: {sv_final}")

# --- End Your Code ---

Circuit - Final Bell State:
     ┌───┐     
q_0: ┤ H ├──■──
     └───┘┌─┴─┐
q_1: ─────┤ X ├
          └───┘

Final Bell Statevector: Statevector([0.70710678+0.j, 0.        +0.j, 0.        +0.j,
             0.70710678+0.j],
            dims=(2, 2))


### 5. Evaluate (Challenge)
The statevector `[0.707, 0, 0, 0.707]` is the $|\Phi^+\rangle$ state. You've successfully created entanglement.

**Challenge:** How would you create the *other* three Bell states?
* $|\Phi^-\rangle = \frac{1}{\sqrt{2}}(|00\rangle - |11\rangle)$
* $|\Psi^+\rangle = \frac{1}{\sqrt{2}}(|01\rangle + |10\rangle)$
* $|\Psi^-\rangle = \frac{1}{\sqrt{2}}(|01\rangle - |10\rangle)$

*Hint: What happens if you apply gates (like `X` or `Z`) to the qubits *before* the Hadamard gate?*

In [ ]:
# Your code here

--- 

## Task 3: Running the CHSH Game

*(Corresponds to Worksheet Part 3)*

### 1. Engage (Why?)
This is the ultimate test. You've seen the theory that a quantum strategy (85.3%) beats any classical strategy (75%). Now, you will code *both* strategies and run a simulation to prove it.

### 2. Explore (Play)
First, let's code the classical game. We need a "referee" function and the simple "always answer 0" strategy from the notes.

In [4]:
# Referee function
def check_win(a, b, x, y):
    """Checks if Alice and Bob win the CHSH game."""
    return (a ^ b) == (x * y) # (a XOR b) == (x AND y)

# Classical strategy: Always answer 0
def classical_strategy(x, y):
    """Alice and Bob both always return 0."""
    a = 0
    b = 0
    return a, b

# --- Run one round of the classical game ---
x_q = 1
y_q = 1
a_c, b_c = classical_strategy(x_q, y_q)
did_win = check_win(a_c, b_c, x_q, y_q)
print(f"Classical Test (x=1, y=1): a={a_c}, b={b_c}. Win? {did_win}")


Classical Test (x=1, y=1): a=0, b=0. Win? False


**Run this code.** This is the one case the classical "always 0" strategy fails. Try other 3 different case. Then, use ``random.randit(0,1)`` to get different case and result.

### 3. Explain (The Concept)
The classical strategy is simple but limited. The quantum strategy uses the entangled Bell state and *changes the measurement basis* (by applying rotation gates) based on the questions `x` and `y`. This allows for stronger-than-classical correlation.

### 4. Elaborate (Apply)
Now, code the quantum strategy. This is the most complex task. We'll build a function that takes `x` and `y` and builds the correct circuit *every time*.

**Recall the strategy:**

* **Alice's angles:** $\theta_A = 0$ (no-op) for $x=0$, $\theta_A = \pi/2$ for $x=1$.
* **Bob's angles:** $\theta_B = \pi/4$ for $y=0$, $\theta_B = -\pi/4$ for $y=1$.

*Implementation Note: To measure in a basis rotated by angle $\theta$, we apply a rotation $R_y(-\theta)$ to the qubit just before the standard Z-basis measurement.*

In [5]:
def quantum_strategy(x, y):
    """Runs one round of the quantum CHSH strategy."""
    
    # 1. Create the Bell state
    qc = QuantumCircuit(2, 2) # Now with 2 classical bits for measurement
    qc.h(0)
    qc.cx(0, 1)
    qc.barrier() # Just to separate preparation from measurement
    
    # 2. Apply rotations based on questions x and y
    # Alice's angles: theta_A = 0 (no-op) for x=0, theta_A = pi/2 for x=1
    if x == 1:
        qc.ry(-pi/2, 0) # Apply Ry(-pi/2) for theta_A = pi/2
        
    # Bob's angles: theta_B = pi/4 for y=0, theta_B = -pi/4 for y=1
    if y == 0:
        qc.ry(-pi/4, 1) # Apply Ry(-pi/4) for theta_B = pi/4
    if y == 1:
        qc.ry(pi/4, 1)  # Apply Ry(pi/4) for theta_B = -pi/4
        
    # 3. Measure
    qc.measure([0, 1], [0, 1])
    
    # 4. Run simulation
    # We need a new simulator instance for each run if we're doing 1 shot
    # Or we can just use the global one
    compiled_circuit = transpile(qc, simulator)
    
    # Run 1 shot (one round of the game)
    job = simulator.run(compiled_circuit, shots=1)
    result = job.result()
    counts = result.get_counts()
    
    # The result is a string like '01' or '11'
    # Get the first (and only) result string
    answer_str = list(counts.keys())[0]
    
    # Qiskit orders bits 'q1 q0'
    b = int(answer_str[0])
    a = int(answer_str[1])
    
    return a, b

# --- Run one round of the quantum game ---
x_q = 1
y_q = 1
a_q, b_q = quantum_strategy(x_q, y_q)
did_win_q = check_win(a_q, b_q, x_q, y_q)
print(f"Quantum Test (x=1, y=1): a={a_q}, b={b_q}. Win? {did_win_q}")

Quantum Test (x=1, y=1): a=0, b=1. Win? True


**Run this code.** This is the case where the classical strategy *always* fails, but the quantum strategy *mostly* wins.

### 5. Evaluate (The Final Test)
Let's run the game 1000 times for each strategy and compare the win rates.

In [6]:
# --- Your Code Here --- 

N_ROUNDS = 1000
classical_wins = 0
quantum_wins = 0

print(f"\nRunning {N_ROUNDS} rounds of the CHSH game...")

for i in range(N_ROUNDS):
    # Get random questions
    x = random.randint(0, 1)
    y = random.randint(0, 1)
    
    # Run classical strategy
    a_c, b_c = classical_strategy(x, y)
    if check_win(a_c, b_c, x, y):
        classical_wins += 1
        
    # Run quantum strategy
    a_q, b_q = quantum_strategy(x, y)
    if check_win(a_q, b_q, x, y):
        quantum_wins += 1

# Calculate win rates
classical_rate = (classical_wins / N_ROUNDS) * 100
quantum_rate = (quantum_wins / N_ROUNDS) * 100

print(f"\n--- Results ---")
print(f"Classical Strategy Win Rate: {classical_rate:.2f}% (Limit: 75%)")
print(f"Quantum Strategy Win Rate:   {quantum_rate:.2f}% (Limit: ~85.3%)")

# --- End Your Code ---


Running 1000 rounds of the CHSH game...

--- Results ---
Classical Strategy Win Rate: 71.10% (Limit: 75%)
Quantum Strategy Win Rate:   86.80% (Limit: ~85.3%)


**Run this final block.** Does your simulation confirm the theory from the notes? You have now experimentally violated the Bell-CHSH inequality.